In [13]:
import pretty_midi
import fluidsynth
from IPython.display import Audio
from IPython import display
import ipywidgets as widgets
import sounddevice as sd
import scipy.io.wavfile as wavfile
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf
import sys

from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

# Sampling rate for audio playback
_SAMPLING_RATE = 44100


In [14]:
# Get a single midi file

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


midi_url = 'https://www.midiworld.com/download/2275'
midi_path = "/Volumes/MAGIC1/CS50/myMusicGen/data/aero.mid"
data_dir = pathlib.Path(midi_path)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'aero.mid',
      origin= midi_url,
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

# The data above contains one Midi file
filenames = str(data_dir)
# print('Number of files:', len(filenames))

# Process a MIDI file
sample_file = filenames
print(sample_file)
# print(sample_file)

# generate a prettyMIDI object for the sample MIDI file
pm = pretty_midi.PrettyMIDI(sample_file)

/Volumes/MAGIC1/CS50/myMusicGen/data/aero.mid


In [15]:

# show tempo

midi_data = pretty_midi.PrettyMIDI(midi_path) # load midi file
print(f"Tempo: {midi_data.estimate_tempo():.2f}") # Print its goal tempo

Tempo: 221.81


In [16]:
# Compute the relative amount of each semitone across the entire song, a proxy for key
'''the .get_chroma() method is used to compute the chromagram of a MIDI file or a specific instrument track within the MIDI file. 
The chromagram represents the intensity of each chroma (musical pitch class) over time.'''
total_velocity = sum(sum(midi_data.get_chroma()))
velocities = [sum(semitone)/total_velocity for semitone in midi_data.get_chroma()]
for i in velocities:
    print(f"{i:.5f}")

0.01969
0.05819
0.22525
0.00077
0.14577
0.01377
0.07136
0.13795
0.00070
0.15646
0.00468
0.16543


In [17]:

# # Change pitch

# # Load MIDI file into PrettyMIDI object
# midi_data = pretty_midi.PrettyMIDI('/Volumes/MAGIC1/CS50/myMusicGen/data/mj.mid')

# # Store the original drum notes
# drum_notes = []
# for instrument in midi_data.instruments:
#     if instrument.is_drum:
#         drum_notes.extend(instrument.notes)

# # Shift pitch of non-drum instruments
# for instrument in midi_data.instruments:
#     if not instrument.is_drum:
#         for note in instrument.notes:
#             note.pitch += 0

# # Create a new instrument track for the preserved drum notes
# drum_instrument = pretty_midi.Instrument(program=0, is_drum=True)
# drum_instrument.notes = drum_notes

# # Add the preserved drum track back to the MIDI data
# midi_data.instruments.append(drum_instrument)

# # Synthesize the resulting MIDI data using sine waves
# audio_data = midi_data.synthesize()

# # Play the audio data
# Audio(data=audio_data, rate=midi_data.synthesize().shape[0] / midi_data.get_end_time())


In [18]:
# Suceeded to extract only drums and meldies separately
# and Both instruments play simultaneously, each coming out of its respective stereo. 
# And can manipulate the pitch for the non-drum insts

# Load MIDI file into PrettyMIDI object
midi_data = pretty_midi.PrettyMIDI(midi_path)

# make an instance of the PrettyMIDI class for drums
# Extract drum MIDI data
drum_midi = pretty_midi.PrettyMIDI()

# make an instance for other insts
melody_midi = pretty_midi.PrettyMIDI()

for instrument in midi_data.instruments:
    if instrument.is_drum:
        drum_midi.instruments.append(instrument)
    else:
        for note in instrument.notes:
            note.pitch -= 0 # Manipulate the pitch of the melodies here. For the original pitch, assign 0
        melody_midi.instruments.append(instrument)

# Convert drum MIDI data to audio
drum_audio = drum_midi.fluidsynth(fs=float(_SAMPLING_RATE))

# convert melody MIDI data to audio
melody_audio = melody_midi.fluidsynth(fs=float(_SAMPLING_RATE))


# # Play the drum audio
# Audio(drum_audio, rate=_SAMPLING_RATE,)

# # Ensure that the length of both audio signals is the same
# min_length = min(len(drum_audio), len(melody_audio)) # It calculates the length of each audio signal using the len() function and compares them using the min() function. The purpose of finding the minimum length is to ensure that both audio signals have the same duration. This is necessary because we want to combine them into a single audio signal and play them simultaneously.
# drum_audio = drum_audio[:min_length] # These lines ensure that both drum_audio and melody_audio are truncated to the minimum length determined in the previous step.
# melody_audio = melody_audio[:min_length] # The [:min_length] slicing operation is used to keep only the samples up to the minimum length for each audio signal. This step is necessary to make sure that both audio signals have the same duration before combining them.

# # Combine the audio signals
# combined_audio = np.vstack((drum_audio, melody_audio))

# # Play the combined audio
# sd.play(combined_audio.T, _SAMPLING_RATE)
# sd.wait()

# Ensure that the length of both audio signals is the same
max_length = max(len(drum_audio), len(melody_audio))
drum_audio = np.pad(drum_audio, (0, max_length - len(drum_audio)))  # We pad the shorter audio signal with zeros using np.pad() so that both signals have the same length.
melody_audio = np.pad(melody_audio, (0, max_length - len(melody_audio))) # We pad the shorter audio signal with zeros using np.pad() so that both signals have the same length.

# Create separate stereo channels for each instrument horizontally. This allows us to create separate stereo channels for each instrument.
drum_audio_stereo = np.column_stack((drum_audio, np.zeros_like(drum_audio)))
melody_audio_stereo = np.column_stack((np.zeros_like(melody_audio), melody_audio))

# Combine the stereo channels
combined_audio = drum_audio_stereo + melody_audio_stereo

# change the playback speed. More details in the note
#  _SAMPLING_RATE = _SAMPLING_RATE * 2

# Play the combined audio
sd.play(combined_audio, _SAMPLING_RATE)
sd.wait()
